In [2]:
import os
import pandas as pd
import random
import subprocess

# Paths
metadata_file = r"D:\DNN\Training_data\fma_metadata\fma_metadata\tracks.csv"
large_music_dir = r"D:\DNN\Training_data\fma_large"
small_music_dir = r"D:\DNN\Training_data\fma_small"
output_dir = r"D:\DNN\Training_data\Music_Mixed_Training"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load metadata
print("Loading metadata...")
df = pd.read_csv(metadata_file, skiprows=[0])  # Ignore first row

df = df[df['subset'].isin(["large", "small"])]  # Keep only large & small subsets

# Extract required columns
df = df[['track_id', 'genre_top', 'subset']]
df = df.dropna(subset=['genre_top'])  # Skip rows with missing genre

df['track_id'] = df['track_id'].astype(int)  # Ensure track_id is an integer


# Function to extract a 2-second clip at a random position
def extract_clip(track_id, genre, subset):
    music_dir = large_music_dir if subset == "large" else small_music_dir
    folder = f"{track_id:06d}"[:3]  # First 3 digits determine subfolder
    input_file = os.path.join(music_dir, folder, f"{track_id:06d}.mp3")
    
    if not os.path.exists(input_file):
        print(f"Missing file: {input_file}")
        return
    
    # Get track duration
    result = subprocess.run(
        ["ffprobe", "-i", input_file, "-show_entries", "format=duration", "-v", "quiet", "-of", "csv=p=0"],
        capture_output=True, text=True
    )
    try:
        duration = float(result.stdout.strip())
    except ValueError:
        print(f"Error reading duration for {input_file}")
        return
    
    if duration < 2:
        print(f"Skipping short track: {input_file}")
        return
    
    # Pick a random start time (ensure at least 2s available)
    start_time = random.uniform(0, duration - 2)
    output_file = os.path.join(output_dir, f"{genre}_{track_id:06d}.wav")
    
    # Extract 2-second clip
    subprocess.run([
        "ffmpeg", "-i", input_file, "-ss", str(start_time), "-t", "2",
        "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "1", output_file,
        "-y"  # Overwrite if needed
    ])
    print(f"Saved: {output_file}")

# Process all tracks
for _, row in df.iterrows():
    extract_clip(row['track_id'], row['genre_top'], row['subset'])

print("Processing complete!")


Loading metadata...


C:\Users\quanz\AppData\Local\Temp\ipykernel_1748\1622195472.py:17: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,48,49,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, skiprows=[0])  # Ignore first row


Saved: D:\DNN\Training_data\Music_Mixed_Training\Hip-Hop_000002.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Hip-Hop_000005.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Pop_000010.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Rock_000135.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Experimental_000137.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Experimental_000138.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Folk_000140.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Folk_000141.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Folk_000142.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Jazz_000144.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Jazz_000145.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Jazz_000146.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Jazz_000147.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Experimental_000148.wav
Saved: D:\DNN\Training_data\Music_Mixed_Training\Experimental